# Table of contents
1. [Data Processing](#processing)
2. [Analytic Models](#models)
    1. [Regression](#reg)
        1. [Model Tuning](#tuning)
        2. [Summary of Models](#summary)
    2. [Identifying Important Features](#ident)
3. [Dimension Reduction and Visualization](#vis)
    1. [Principal Component Analysis](#pca)
    2. [t-Stochastic Nearest Neighbor Embedding (t-SNE)](#tsne)


In [1]:
import math
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls

## Data Processing <a name="processing"></a>
The conversion from Excel Workbook to CSV was done with my old code. The dataset consists of 991 examples with 28 features, and the risk factor of an employee, which we treat as the label. First, we load the CSV file into a numpy array and sort the data by risk factor from low to high. Then, separate the features and the labels (risk factor). Since the features are all on different scales, we'll standardize the entire dataset.

In [3]:
from sklearn import preprocessing

file = np.loadtxt(open("data/safeway/report.csv", "rb"), delimiter=",")
# Sort data low-to-high based on risk factor
file_sorted = file[file[:,0].argsort()]
# Split the features from the labels
data = file_sorted[:,1:] # Features
labels = file_sorted[:,0] # Labels

# Sanity check
print("Dimensions of the data array:",data.shape)
print("Dimensions of the labels array:",labels.shape)
data = preprocessing.scale(data)
print("Dimensions of the data array after preprocessing:",data.shape)

hist = [go.Histogram(x=labels)]
layout = go.Layout(
    title='Distribution of Risk Factors within Dataset',
    xaxis=dict(
        title='Risk Factor'
    ),
    yaxis=dict(
        title='Count'
    ),
)
py.iplot(go.Figure(data=hist, layout=layout), filename='Risk_Facotr_Histogram')

Dimensions of the data array: (991, 28)
Dimensions of the labels array: (991,)
Dimensions of the data array after preprocessing: (991, 28)


In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train-test sets, 90% train and 10% test
X_train, X_test, y_train_reg, y_test_reg = train_test_split(data, labels, test_size=0.1, random_state=15)

## Analytic Models <a name="models"></a>
### Regression Models <a name="reg"></a>
We use a variety of regression models to predict the numeric risk factor given new features. For performance evaluation, we report the mean squared error, mean absolute error, and R^2 score on the test set for each algorithm. Mathematically, MSE is the (prediction - true value)^2, MAE is abs(prediction - true value), and R^2 is the proportion of the variance in the dependent variable that is predictable from the independent variables. Specifically, we apply Linear/Ridge/Lasso regression, SVM, k-NN, Decision Tree, Random Forest, and Multilayer Perceptron algorithms to this dataset. Some of these models need to be tuned for better performance. 

### Model Tuning/Hyperparameter Search <a name="tuning"></a>
For ridge and lasso regression, there is a hyperparameter alpha that needs to be tuned. The larger the value of alpha, the greater the amount of shrinkage and thus the coefficients become more robust to collinearity. For k-NN, we can try different numbers of neighbors the algorithm considers when making prediction.

In [5]:
from sklearn import linear_model, svm, tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Returns MSE, MAE, and R2 score of the model on test set
# Parameters:
# name: String containing name of the model
# model: Object to perform regression with
def evaluate_model(name, model):
    names, mse, mae, r2 = ([] for i in range(4))
    names.append(name)
    model.fit(X_train, y_train_reg)
    predicted = model.predict(X_test)
    
    mse.append('{0:.4}'.format(mean_squared_error(y_test_reg, predicted)))
    mae.append('{0:.4}'.format(mean_absolute_error(y_test_reg, predicted)))
    r2.append('{0:.4}'.format(r2_score(y_test_reg, predicted)))
    return [mse[0], mae[0], r2[0]]

# Generates a plot of the tuning curve for MSE, MAE, and R2
# Parameters:
# name: String containing name of the model
# hp_name: String containing the name of the hyperparameter being tuned
# hp_values: List of values that the hyperparameter can take on
def plot_tuning_curve(name, hp_name, hp_values):
    scores = []
    # Vary Alphas on a Log-Scale
    for a in hp_values:
        if name == "Ridge Regression":
            model = linear_model.Ridge(alpha=a)
        if name == "Lasso Regression":
            model = linear_model.Lasso(alpha=a)
        if name == "k-NN":
            model = KNeighborsRegressor(n_neighbors=a)
        if name == "MLP":
            model = MLPRegressor(activation ='logistic', hidden_layer_sizes=(5, ), max_iter=50000, solver='lbfgs', alpha=a)
        scores.append(evaluate_model("%s (a=%s)" % (name, a), model))
        
    scores = np.array(scores)
    
    mse = go.Scatter(
        x = hp_values,
        y = scores[:,0],
        mode = 'lines+markers',
        name = 'MSE'
    )

    mae = go.Scatter(
        x = hp_values,
        y = scores[:,1],
        mode = 'lines+markers',
        name = 'MAE'
    )

    r2 = go.Scatter(
        x = hp_values,
        y = scores[:,2],
        mode = 'lines+markers',
        name = 'R2'
    )

    layout = go.Layout(
        title='Tuning %s Varying %s' % (name, hp_name),
        xaxis=dict(
            title='%s (Log-Scale)' % hp_name,
            type='log'
        ),
        yaxis=dict(
            title='Value'
        ),
    )
    scatter = [mse, mae, r2]
    return go.Figure(data=scatter, layout=layout)

In [6]:
py.iplot(plot_tuning_curve("Ridge Regression", "Alpha", [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500]), filename='Ridge_Regression_Alpha_Tuning')

In [7]:
py.iplot(plot_tuning_curve("Lasso Regression", "Alpha", [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500]), filename='Lasso_Regression_Alpha_Tuning')

In [8]:
py.iplot(plot_tuning_curve("k-NN", "Number of Neighbors", [1, 5, 10, 32, 50, 100, 500]), filename='kNN_Tuning')

In [9]:
py.iplot(plot_tuning_curve("MLP", "Alpha", [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000]), filename='MLP_Alpha_Tuning')

### Summary of Models <a name="summary"></a>

In [10]:
# Report the MSE, MAE, and R2 values for all models
names = np.array(["Linear Regression", "Lasso Regression", "Ridge Regression", "SVM RBF Kernel","SVM Linear Kernel", "k-NN", "Decision Tree", "Random Forest", "Multilayer Perceptron"])
report = []

# Linear regression/ordinary least squares
linreg = linear_model.LinearRegression()
report.append(evaluate_model("Linear Regression", linreg))

# Lasso Regression, aka L1 regularized
lasso = linear_model.Lasso(alpha=0.1)
report.append(evaluate_model("Lasso Regression", lasso))

# Ridge Regression, aka L2 regularized
ridge = linear_model.Ridge(alpha=50)
report.append(evaluate_model("Ridge Regression", ridge))

# Support Vector Regressor
svr = svm.SVR(kernel='rbf')
report.append(evaluate_model("SVM RBF Kernel", svr))

# Support Vector Regressor
lsvr = svm.LinearSVR()
report.append(evaluate_model("SVM Linear Kernel", lsvr))

# k-Nearest Neighbors using k=32 (usually use sqrt(number of examples))
knn = KNeighborsRegressor(n_neighbors=10)
report.append(evaluate_model("k-NN (32)", knn))

# Decision Tree
dt = tree.DecisionTreeRegressor()
report.append(evaluate_model("Decision Tree", dt))

# Random Forest
rf = RandomForestRegressor(max_depth=5)
report.append(evaluate_model("Random Forest", rf))

# Perceptron
mlp = MLPRegressor(activation ='logistic', hidden_layer_sizes=(5, ), max_iter=50000, solver='lbfgs', alpha=0.1)
report.append(evaluate_model("Multilayer Perceptron", mlp))

report = np.array(report)

evaluations = {'Models': names, 'MSE': report[:,0], 'MAE': report[:,1], 'R2': report[:,2]}
df = pd.DataFrame(evaluations)[['Models', 'MSE', 'MAE', 'R2']]

table = ff.create_table(df)
py.iplot(table, filename='Regression_Comparison')

### Identifying Important Features <a name="ident"></a>
Since this data is high dimensional, it's best to identify which features are more important for the predictor. One way to identify the important features is by looking at the coefficients learned by linear models (linear, lasso, and ridge regression). Intuitively, coefficients of features that have larger magnitude are more important.

In [11]:
feature_descriptions = ["Total Number Of Transactions","Total Items Count","Total Sales Amount","Coupons outside of orders","Coupons - Highest Number Of Uses Of Promotion Code","Coupons - Percentage Of Total Amount","Coupons Total Amount","Coupons Count","","Coupons Average Amount","Coupons - Percentage Of All Transactions","Refunds Trx Count","Refunds Item Count","Refunds Total Amount","Department Refunds Percentage Of Total Amount","Department Refunds Total Amount","Department Refunds Percentage Of Items","Department Refunds Item Count","Item Voids Item Count","Item Voids Total Amount Voided","Department Sales Percentage Of All Items","Department Sales Item Count","Department Sales Total Amount","Zero Trx Percentage","Zero Count","Zero Trx With Alcohol","Payment Card Relationship Max No Of Card Uses","No Sales Per Day","Base Avg Basket Size"]

In [12]:
lin_c = np.absolute(linreg.coef_).argsort()
lasso_c = np.absolute(lasso.coef_).argsort()
ridge_c = np.absolute(ridge.coef_).argsort()
print("Linear regression:", lin_c)
print("Lasso regression:", lasso_c)
print("Ridge regression:", ridge_c)
#print("Random Forest:", rf.feature_importances_.argsort())

Linear regression: [21 17  8 24 22  5  0 12 20 25 16 14 13 18 15 26  4 19 10 23 11  9 27  3  7
  6  2  1]
Lasso regression: [27 18 15 26 12 22 20  8 21 24  5  4  2 23 14  1 19 17 25 10 16  9  0 13 11
  3  6  7]
Ridge regression: [22  8 21  4  5 12 17 24 18 20 14 15 16 26 25  2 19 23 27 13 10  0 11  9  1
  7  3  6]


We can actually do better than just using the coefficients. Recursive feature elimination will train a model on subsets of the features by pruning the least important features determined by the model's coefficients. This recursive process will help us narrow down the top N features.

In [13]:
# Recursive feature elimination
from sklearn.feature_selection import RFE, RFECV

rfe = RFE(linreg, n_features_to_select=2)
rfe.fit(data, labels)
linreg_rfe = rfe.ranking_.argsort()

rfe = RFE(lasso, n_features_to_select=2)
rfe.fit(data, labels)
lasso_rfe = rfe.ranking_.argsort()

rfe = RFE(ridge, n_features_to_select=2)
rfe.fit(data, labels)
ridge_rfe = rfe.ranking_.argsort()

rfe = RFE(lsvr, n_features_to_select=2)
rfe.fit(data, labels)
svr_rfe = rfe.ranking_.argsort()

rfe = RFE(dt, n_features_to_select=2)
rfe.fit(data, labels)
dt_rfe = rfe.ranking_.argsort()

rfe = RFE(rf, n_features_to_select=2)
rfe.fit(data, labels)
rf_rfe = rfe.ranking_.argsort()

Now, we plot the top 2 ranking features selected by RFE for each of the regression models, decision tree, and random forest.

In [14]:
# Parameters:
# name: String containing name of the model
# rfe_array: Ranking of features produced by RFE
# Return scatter plot of the top two ranked features
def generate_RFE_plot(name, rfe_array):
    trace1 = go.Scatter(
        x = data[:,rfe_array[0]],
        y = data[:,rfe_array[1]],
        mode='markers',
        marker=dict(
            size='7',
            opacity=0.75,
            color = labels, #set color equal to a variable
            colorscale='Portland',
            showscale=True,
            line = dict(
                width = 0.5
            )
        ),
        text=["Risk: %s" %l for l in labels]
    )

    layout= go.Layout(
        title= 'Top 2 Features Ranked by RFE for '+name,
        hovermode= 'closest',
        xaxis= dict(
            title= feature_descriptions[rfe_array[0]]
        ),
        yaxis=dict(
            title= feature_descriptions[rfe_array[1]]
        )
    )

    fig = go.Figure(data=[trace1], layout=layout)
    return fig

In [15]:
py.iplot(generate_RFE_plot("Linear Regression", linreg_rfe), filename='Top 2 Features Ranked by RFE for Linear Regression')

In [16]:
py.iplot(generate_RFE_plot("Lasso Regression", lasso_rfe), filename='Top 2 Features Ranked by RFE for Lasso Regression')

In [17]:
py.iplot(generate_RFE_plot("Ridge Regression", ridge_rfe), filename='Top 2 Features Ranked by RFE for Ridge Regression')

In [18]:
py.iplot(generate_RFE_plot("SVM", svr_rfe), filename='Top 2 Features Ranked by RFE for Support Vector Regression')

In [19]:
py.iplot(generate_RFE_plot("Random Forest", rf_rfe), filename='Top 2 Features Ranked by RFE for Random Forest')

## Dimension-Reduction and Visualization <a name="vis"></a>
### Principal Component Analysis <a name="pca"></a>
Using feature selection/RFE, we lose out on potentially a lot of relationships between the many features. Another way to represent our data in two or three dimensions is by using PCA. Sometimes using only the principal components will have more predictive power for the model, so we try linear regression with the top 2 PC.

In [20]:
# Run PCA on data to reduce to 2 components
pca = PCA(n_components=2)
pca.fit(data)
data_pca = pca.transform(data)

# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(data_pca, labels, test_size=0.1, random_state=15)

# Train Linear Regression
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
predicted = model.predict(X_test)

MSE = mean_squared_error(y_test, predicted)
MAE = mean_absolute_error(y_test, predicted)
R2 = r2_score(y_test, predicted)

print("Linear Regression Results with PCA")
print("Mean squared error: %.2f" % MSE)
print("Mean absolute error: %.2f" % MAE)
print('Coefficient of determination: %.2f' % R2)

Linear Regression Results with PCA
Mean squared error: 2.18
Mean absolute error: 1.17
Coefficient of determination: -0.01


In [21]:
# Plot PCA points
trace1 = go.Scatter(
    x = data_pca[:,0],
    y = data_pca[:,1],
    mode='markers',
    marker=dict(
        size='7',
        opacity=0.75,
        color = labels, #set color equal to a variable
        colorscale='Portland',
        showscale=True,
        line = dict(
            width = 0.5
        )
    ),
    text=["Risk: %s" %l for l in labels]
)

layout= go.Layout(
    title= 'Projection of data using first two prinicpal components',
    hovermode= 'closest',
    xaxis= dict(
        title="PC 1"
    ),
    yaxis=dict(
        title= "PC 2"
    )
)

fig = go.Figure(data=[trace1], layout=layout)
py.iplot(fig, filename='PCA_2')

In [22]:
pca = PCA(n_components=3)
pca.fit(data)
data_pca3 = pca.transform(data)

trace1 = go.Scatter3d(
    x = data_pca3[:,0],
    y = data_pca3[:,1],
    z = data_pca3[:,2],
    mode='markers',
    marker=dict(
        size='3',
        opacity=0.75,
        color = labels, #set color equal to a variable
        colorscale='Portland',
        showscale=True,
    ),
    text=["Risk: %s" %l for l in labels]
)

layout= go.Layout(
    title= 'Projection of data using first three prinicpal components',
    hovermode= 'closest',
    xaxis= dict(
        title="PC 1"
    ),
    yaxis=dict(
        title= "PC 2"
    )
)

fig = go.Figure(data=[trace1], layout=layout)
py.iplot(fig, filename='PCA_3')

### t-Stochastic Nearest Neighbor Embedding (t-SNE) <a name="tsne"></a>
Another method to visualize high dimensional data is by using data embedding techniques (PCA and LDA are special cases of these methods). A method that works particularly well on images and have previously been shown to work better than PCA for biology data is t-SNE.

The main challenge with t-SNE and PCA visualizations on this dataset is assigning an objective evaluation of how well the resulting visualizations are. We can calculate the silhouette score, which is used for evaluating clustering algorithms. To do so, we bin our data into N different classes. First, we run k-Means on the original dataset to determine the best number of classes to use. We try a range of k values, and choose the best silhouette score.

In [23]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

range_n_clusters = range(2,10)
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = kmeans.fit_predict(data)
    silhouette_avg = silhouette_score(data, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)    

For n_clusters = 2 The average silhouette_score is : 0.718650809074
For n_clusters = 3 The average silhouette_score is : 0.179101457472
For n_clusters = 4 The average silhouette_score is : 0.133338710476
For n_clusters = 5 The average silhouette_score is : 0.13100932328
For n_clusters = 6 The average silhouette_score is : 0.138889684897
For n_clusters = 7 The average silhouette_score is : 0.141661654502
For n_clusters = 8 The average silhouette_score is : 0.143445754489
For n_clusters = 9 The average silhouette_score is : 0.106036357711


The best N to use is 2 (i.e. binary classification: risky or not). We split the data into two bins, and then use t-SNE to transform the dataset. Lastly, we compare the resulting embedding with the assigned classes and compute the silhouette score.

In [24]:
# Originally I thought to use 5 classes, but binary classification will work better
#n, bins, patches = plt.hist(labels, 5)
#class_dict = ["Low", "Low-Medium", "Medium", "Medium-High", "high"]
#class_labels = np.array([])
#for i in range(n.shape[0]):
#    labels_ = np.full(int(n[i]), class_dict[i])
#    class_labels = np.concatenate((class_labels,labels_))
#print(class_labels.shape)

labels_low = np.full(496, "Low")
labels_high = np.full(495, "High")
class_labels = np.concatenate((labels_low, labels_high))
print(class_labels.shape)

(991,)


Since the t-SNE cost function is non-convex, we will get different results from different initializations. Therefore, we have to tune the perplexity and learning rate parameters. Additionally, running it for longer iterations will also result in different embeddings. We can also be selective about the number of features we use, so we also try the top N features produced from RFE for random forest. We then tune the model based on its silhouette score.

In [ ]:
# Warning: Takes up to 12 hours to run
'''
from sklearn.metrics import silhouette_score

perplexity = [2, 5, 30, 50, 100]
learning_rate = [10, 100, 1000]
iters = [250, 500, 1000, 2000]
features = rf_rfe # Use random forest RFE ranking
hp_dict_2d_fe = {}
j = 0
for num_features in range(3, len(features)+1):
    new_data = data[:,features[0]][:,np.newaxis]
    for i in range(1,num_features):
        new_data = np.hstack((new_data, data[:,features[i]][:,np.newaxis]))
    for p in perplexity:
        for lr in learning_rate:
            for i in iters:
                print(j)
                j += 1
                data_embedded = TSNE(n_components=2,perplexity=p,learning_rate=lr,n_iter=i).fit_transform(new_data)
                score = silhouette_score(data_embedded, class_labels)
                hp = str(num_features)+"_"+str(p)+"_"+str(lr)+"_"+str(i)
                hp_dict_2d_fe[hp] = score
                plt.figure(figsize=(15,15))
                plt.scatter(data_embedded[:,0], data_embedded[:,1], c=labels, cmap=cm.hot_r, s=50*np.ones(991), alpha=0.5)
                plt.colorbar()
                plt.savefig("figures_feature_elim/"+hp+".png", bbox_inches='tight')
                plt.close()
np.save("hp_dict_2d_fe.npy", hp_dict_2d_fe)
'''

In [25]:
from collections import Counter

# Since I saved the dictionary with numpy, it's loaded as a feature array rather than a dictionary. 
# So I manually it convert it back here.
hp_dict = np.load("hp_dict_2d_fe.npy")
new_dict = {}
for key in hp_dict.item():
    new_dict[key] = hp_dict.item().get(key)

In [26]:
# todo: turn this into a dataframe/plotly table
best = Counter(new_dict)
print("Top 10 t-SNE hyperparameter settings and silhouette scores")
for pair in best.most_common(10):
    print(pair)
print("Compared against PCA silhouette score")
print(("PCA", silhouette_score(data_pca, class_labels)))

Top 10 t-SNE hyperparameter settings and silhouette scores
('4_5_10_250', 0.14942884)
('3_30_10_1000', 0.13991536)
('3_30_1000_500', 0.13929713)
('3_30_100_500', 0.1388244)
('3_30_10_2000', 0.13852572)
('3_30_10_500', 0.13746214)
('3_50_10_500', 0.13647822)
('3_5_100_250', 0.13642122)
('3_30_100_1000', 0.13638541)
('3_30_1000_1000', 0.13621932)
Compared against PCA silhouette score
('PCA', 0.071961015092428415)


Next, we plot the top 5 t-SNE embeddings, and run k-means (k=2) to identify the clusters in the embeddings for a full visualization.

In [27]:
def tsne_with_centroids(hp_setting, feature_rankings):
    new_data = data[:,feature_rankings[0]][:,np.newaxis]
    for i in range(1,int(hp_setting[0])):
        new_data = np.hstack((new_data, data[:,feature_rankings[i]][:,np.newaxis]))
    data_embedded = TSNE(random_state=10, n_components=2,perplexity=int(hp_setting[1]),learning_rate=int(hp_setting[2]),n_iter=int(hp_setting[3])).fit_transform(new_data)
    kmeans = KMeans(n_clusters=2, random_state=10).fit(data_embedded)
    centers = np.array(kmeans.cluster_centers_)
    
    tsne = go.Scatter(
        name = "Data Embedding",
        x = data_embedded[:,0],
        y = data_embedded[:,1],
        mode='markers',
        marker=dict(
            size='7',
            opacity=0.75,
            color = labels, #set color equal to a variable
            colorscale='Portland',
            showscale=True,
            line = dict(
                width = 0.5
            )
        ),
        text=["Risk: %s" %l for l in labels]
    )
    centroids = go.Scatter(
        name = "Centroids",
        x = centers[:,0],
        y = centers[:,1],
        mode='markers',
        marker=dict(
            size='15',
            symbol= "x",
            color = "black"
            )
    )
    layout= go.Layout(
        title= 't-SNE Embedding using %s Features, Perplexity=%s, Learning Rate=%s, Iters=%s' % tuple(hp_setting),
        hovermode= 'closest',
    )

    fig = go.Figure(data=[tsne, centroids], layout=layout)
    return fig

In [28]:
py.iplot(tsne_with_centroids("4_5_10_250".split("_"),rf_rfe), filename="4_5_10_250")

In [30]:
py.iplot(tsne_with_centroids("3_30_10_1000".split("_"),rf_rfe), filename="3_30_10_1000")

In [31]:
py.iplot(tsne_with_centroids("3_30_1000_500".split("_"),rf_rfe), filename="3_30_1000_500")

In [33]:
py.iplot(tsne_with_centroids("3_30_100_500".split("_"),rf_rfe), filename="3_30_100_500")

In [34]:
py.iplot(tsne_with_centroids("3_30_10_2000".split("_"),rf_rfe), filename="3_30_10_2000")

Feature selection is important for visualization. Here, we plot the average silhouette score for t-SNE embeddings across different hyperparameter settings varying the number of features used. As more features are used, the lower the silhouette score.

In [35]:
# Find the mean silhouette score across number of features used
mean = {}
for i in range(3,29):
    results = [v for k,v in new_dict.items() if k.startswith(str(i))]
    mean[i] = np.mean(np.array(results))

mean = go.Scatter(
    x = list(mean.keys()),
    y = list(mean.values()),
    mode = 'lines+markers',
)

layout = go.Layout(
    title='Mean Silhouette Score Across Number of Features Used',
    xaxis=dict(
        title='Number of Features',
    ),
    yaxis=dict(
        title='Mean Silhouette Score'
    ),
)
scatter = [mean]
fig = go.Figure(data=scatter, layout=layout)
py.iplot(fig, filename='Mean_Silhouette_Score')

# Exclude from paper (just for exploration)
## Classification Models
I tried treating this problem as a classification task, just to see how well it would perform. Turns out regression is definitely more suitable.

### Assigning Classes/Labels to Examples
First, assign each example into a class to perform classification later. For the risk factors, min is 4.57 and max is 16, and we split it evenly into 5 bins using a histogram (AKA equal-width binning).

In [ ]:
# Generate histogram of the data, split into 5 bins
n, bins, patches = plt.hist(labels, 5, facecolor='green', alpha=0.75)
print("Number of elements per bin:", n) # Number of elements in each bin

plt.ylabel('Counts')
plt.xlabel('Risk Factor')
plt.show()

The data is poorly distributed if we split it evenly based on width. So now, we try equal-frequency binning to split into 5 bins with the same number of elements each.

In [ ]:
class_labels = ["Low", "Low-Medium", "Medium", "Medium-High", "high"]
lows = np.full(991//5+1, class_labels[0])
lms = np.full(991//5, class_labels[1])
meds = np.full(991//5, class_labels[2])
mhs = np.full(991//5, class_labels[3])
highs = np.full(991//5, class_labels[4])

classes = np.concatenate((lows,lms,meds,mhs,highs))
print("Dimensions of the classes array:", classes.shape) # Sanity check to make sure still 991

n, bins, patches = plt.hist(classes, 5, facecolor='green', alpha=0.75)
print("Number of elements per bin:", n) # Number of elements in each bin
plt.show()

For classification, we train an SVM using linear (parametric) and RBF (non-parametric) kernels on the data. For performance evaluation, we output the test set's total precision, recall, and F1 score, and for each class.

In [ ]:
# Split the data into train-test sets, 90% train and 10% test
X_train, X_test, y_train_class, y_test_class = train_test_split(data, classes, test_size=0.1, random_state=15)

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report

# Linear Support Vector Classifier
model = LinearSVC()
model.fit(X_train, y_train_class)
predicted = model.predict(X_test)

print(classification_report(y_test_class, predicted))

In [ ]:
# RBF Support Vector Classifier
model = SVC()
model.fit(X_train, y_train_class)
predicted = model.predict(X_test)

print(classification_report(y_test_class, predicted))